# 3-1. 들어가며


안녕하세요. 오늘은 단어의 빈도를 이용해서 텍스트를 표현하는 방법들과 텍스트의 분포를 이용해서 텍스트를 토큰화하는 방법에 대해서 배워보겠습니다. 오늘 다루게 되는 방법들은 Word2Vec 같은 임베딩을 활용한 분산 표현을 학습하는 기법이 대중화되기 이전에 많이 쓰이던 방법들입니다. 하지만 텍스트의 분포를 활용해서 텍스트를 벡터화하는 아이디어의 발전사를 통해 기초적인 개념을 잘 다져볼 수 있을 것입니다.

## 목차

1. 단어 빈도를 이용한 벡터화<br>
(1) Bag of Words<br>
(2) Bag of Words 구현해보기<br>
(3) DTM과 코사인 유사도<br>
(4) DTM의 구현과 한계점<br>
(5) TF-IDF<br>
(6) TF-IDF 구현하기<br>

2. LSA와 LDA<br>
(1) LSA<br>
(2) LSA 실습<br>
(3) LDA<br>
(4) LDA 실습<br>

3. 텍스트 분포를 이용한 비지도 학습 토크나이저<br>
(1) 형태소 분석기와 단어 미등록 문제<br>
(2) soynlp<br>

## 준비물

개인 실습환경에 맞추어 경로를 변경, 디렉토리를 생성해 주세요.
```bash
$ mkdir -p ~/aiffel/topic_modelling/data
```

# 3-2. 단어 빈도를 이용한 벡터화 (1) Bag of Words

머신러닝 모델은 기계이기 때문에, 텍스트보다는 수치화된 숫자를 더 잘 처리할 수 있습니다. 이런 이유로 기계가 자연어 처리를 원활히 할 수 있도록, 자연어 처리에서는 전처리 과정에서 텍스트를 숫자 벡터로 변환하는 **벡터화(Vectorization)** 라는 과정을 거치게 됩니다.

벡터화 방법으로는 크게 **(1) 통계와 머신 러닝을 활용한 방법**, 그리고 **(2) 인공 신경망을 활용하는 방법** 두 가지로 나눠볼 수 있는데요. 인공 신경망을 활용하는 방법은 다음에 배워보도록 하고, 오늘은 전자의 방법을 통해 벡터화하는 방법에 대해서 배워보겠습니다.

## Bag of Words

**Bag of Words**는 자연어 처리(Natural Language Processing)나 정보 검색(Information Retrieval)에서 쓰이는 매우 간단한 단어 표현 방법으로, 문서 내의 단어들의 분포를 보고 이 문서의 특성을 파악하는 기법입니다. 줄여서 간단히 BoW라고 표기하기도 합니다.

아직은 설명이 아리송하지요? 조금 더 쉽게 설명해볼게요. Bag of Words는 직역하면 "단어들의 가방"입니다. Bag of Words는 문서를 단어들의 가방으로 가정합니다.

<p align="center"><img src=https://d3s0tskafalll9.cloudfront.net/media/images/GN-2-L-1.max-800x600.png></p>

어떤 한 문서에 등장하는 텍스트를 전부 단어 단위로 토큰화(tokenization)를 합니다. 그러면 이제 수많은 단어들이 나올 텐데, 이 단어들을 전부 가방에 집어넣습니다. 그리고 이 가방을 마구 흔들어요. 그렇게 되면 단어들의 순서는 무시되고 전부 섞이게 되겠지요?

그리고 문서에 사과라는 단어가 6번 등장했다면, 이 가방에는 사과라는 단어가 6개 있을 것입니다. Bag of Words는 중복을 제거하지 않고, 단어들을 카운트해요. 결론적으로 Bag of Words는 단어들의 순서는 무시하고, 단어들의 빈도 정보는 그대로 보존해요.

예를 들어볼게요! 다음과 같은 `doc1`이 있다고 해볼게요.
```python
doc1 = 'John likes to watch movies. Mary likes movies too.'
```

이를 BoW로 표현하면 아래와 같습니다.
```python
BoW1 = {"John":1, "likes":2, "to":1, "watch":1, "movies":2, "Mary":1, "too":1}
```
아래와 같이 `doc2`가 있다고 해보겠습니다.
```python
doc2 = 'Mary also likes to watch football games.'
```

이를 BoW로 표현하면 아래와 같겠군요!
```python
BoW2 = {"Mary":1, "also":1, "likes":1, "to":1, "watch":1, "football":1, "games":1}
```

위의 BoW에서 각 key는 단어, 각 value는 각 단어가 주어진 문서에 등장한 횟수에요. 이때, 문서 내 단어의 순서는 중요하지 않습니다. 그렇기 때문에 아래의 `BoW`는 `BoW1`과 본질적으로 동일합니다.
```python
BoW = {"too":1, "Mary":1, "movies":2, "John":1, "watch":1, "likes":2, "to":1}
BoW1 = {"John":1, "likes":2, "to":1, "watch":1, "movies":2, "Mary":1, "too":1}
```

BoW 방식으로 문장을 벡터화한다면 `I ate lunch.` 와 `Lunch ate I.` 는 같은 문장이 됩니다. 이렇게 어순에 따라 달라지는 의미를 반영하지 못한다는 점에서 BoW는 한계를 가집니다.

이번에는 두 문서를 합쳐서 새로운 문서를 정의하고 BoW를 구해볼게요!
```python
doc3 = 'John likes to watch movies. Mary likes movies too. Mary also likes to watch football games.'
```

이로부터 만들어진 `BoW3`은 아래와 같습니다.
```python
BoW3 = {"John":1, "likes":3, "to":2, "watch":2, "movies":2, "Mary":2, "too":1, "also":1, "football":1, "games":1};
```

# 3-3. 단어 빈도를 이용한 벡터화 (2) Bag of Words 구현해보기

아래에서는 각각 텐서플로우 케라스와 사이킷런 라이브러리를 사용해 Bag of Words를 만들겠습니다.

## keras Tokenizer 활용

케라스 토크나이저를 통해 Bag of Words를 만들어 보겠습니다.

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


여기서 **단어장(Vocabulary)** 이라는 또 다른 개념을 이해해 보겠습니다. 문헌에 따라 사전 또는 단어 집합이라고 불리는데요. 단어장이란 중복을 제거한 단어들의 집합을 말합니다. 이는 Bag of Words랑은 다른 개념입니다. 위의 예제에서는 단어장의 크기가 10개네요!

## scikit-learn CountVectorizer 활용

Bag of Words를 만드는 또 다른 방법은 사이킷런의 `CountVectorizer`를 사용하는 것입니다.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


`CounterVectorizer`의 `vector.fit_transform(입력 문장).toarray()`으로 출력되는 Bag of Words 결과를 보면 각 단어의 빈도만 출력될 뿐, 어떤 단어의 빈도인지는 나오지 않습니다. `[[1 1 1 1 3 2 2 2 1 2]]`이라는 값만으로는 각 숫자가 어떤 단어의 빈도인지 알 수가 없지요?

이는 `vector.vocabulary_`를 통해서 각 단어에 부여된 인덱스를 확인하면 되는데요. 주의할 점은 이 인덱스는 0부터 시작한다는 점입니다. 가령, `also`의 인덱스는 0이네요. 그런데 BoW에서 0번째 위치한 값은 1이니까, `also`의 빈도는 1입니다. 또는 `likes`의 인덱스는 4인데, BoW에서 4번째 위치한 값은 3이니까 `likes`의 빈도는 3임을 알 수 있지요!

그리고 이 Bag of Words의 단어장의 크기는 아래와 같이 구하면 되겠지요!

In [3]:
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(Vocabulary)의 크기 : 10


단어장이라는 개념은 Bag of Words를 위해 사용되는 개념이라기보다는, 자연어 처리에서 보편적으로 쓰이는 용어입니다. 그런데 이를 계속 강조하는 이유는 지금부터 사용할 **DTM(Document-Term Matrix)** 이라는 개념 이해를 돕기 위해서입니다. 지금 Bag of Words를 배우면서 이런 생각이 들지 않으셨나요? 그래서 이 개념을 도대체 어디서 사용할 수 있다는 거지? 이를 이해하기 위해서 DTM으로 가보겠습니다!

# 3-4. 단어 빈도를 이용한 벡터화 (3) DTM과 코사인 유사도

**DTM(Document-Term Matrix)** 은 직역하면 문서-단어 행렬입니다. DTM은 여러 문서의 Bag of Words를 하나의 행렬로 구현한 것인데요. 다시 말해 DTM은 각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합시킵니다. DTM은 문서를 행으로, 단어를 열로 가지는 행렬이지만, 문헌에 따라서는 열을 문서로 하고 단어를 행으로 하여 **TDM(Term-Document Matrix)** 이라고 부르기도 합니다.

예를 들어, 아래와 같은 3개의 문서가 있다고 해보겠습니다.
> Doc 1: Intelligent applications creates intelligent business processes<br>
Doc 2: Bots are intelligent applications<br>
Doc 3: I do business intelligence<br>

이 3개의 문서로부터 얻을 수 있는 DTM은 아래와 같습니다.

<p align="center"><img src=https://d3s0tskafalll9.cloudfront.net/media/images/GN-2-L-2.max-800x600.png></p>
<center>[출처 : https://www.darrinbishop.com/blog/2017/10/text-analytics-document-term-matrix/]</center>

위의 DTM을 보면 각 행은 각 문서를 나타내고 있습니다. 그리고 각 열은 문서 3개의 통합 단어장(Vocabuary)에 있는 단어들로 구성되어져 있습니다. 이렇게 구성하면 앞서 문서 1개로 BoW를 만들었을 때와는 달리, 각 행은 많은 값이 0으로 구성이 됩니다.

이때 각 행을 문서 벡터(document vector), 열을 단어 벡터(word vector)라고 부를 수 있는데요. 문서의 수가 많아지면 많아질수록, 통합 단어장의 크기도 커지게 되어서 DTM은 결국 문서 벡터와 단어 벡터 모두 대부분의 값이 0이 되는 성질을 가지고 있습니다.

여러 문서들로 DTM을 만들었다면, 이제 Bag of Words로 무엇을 할 수 있는지를 이해할 차례입니다. DTM을 사용하면 각 문서들을 비교하고, 각 문서 간 유사도를 구할 수 있게 됩니다. 예를 들어볼까요?

>문서1 : I like dog<br>
문서2 : I like cat<br>
문서3 : I like cat I like cat<br>

위와 같이 3개의 문서가 있다고 해볼게요. 각 단어에 대한 인덱스를 `cat`은 0, `dog`는 1, I는 2, `like`는 3이라고 했을 때 생성되는 DTM의 형태는 아래와 같습니다.

<p align="center"><img src=https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-2-L-3.png></p>

이제 각 문서 벡터의 유사도를 구해보겠습니다. 저자는 가장 보편적으로 사용되는 유사도 계산 방법인 **코사인 유사도**를 택했습니다. 아래와 같이 각 문서 벡터와 코사인 유사도 함수를 정의합니다.
